In [ ]:
import os
from pathlib import Path
import slicer


# taken from https://discourse.slicer.org/t/converting-stl-files-to-binary-label-maps-in-nii-format-using-python/13038/9
def stl_to_nifti(stl_path, reference_volume_path, output_path):
    referenceVolumeNode = slicer.util.loadVolume(str(reference_volume_path))
    segmentationNode = slicer.util.loadSegmentation(str(stl_path))
    outputLabelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass(
        "vtkMRMLLabelMapVolumeNode"
    )
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(
        segmentationNode, outputLabelmapVolumeNode, referenceVolumeNode
    )
    slicer.util.saveNode(outputLabelmapVolumeNode, str(output_path))


def convert_segmentations(stl_dir, img_dir, out_dir):
    filenames = os.listdir(str(stl_dir))
    for i, fname in enumerate(filenames):
        print(f"Converting {i}/{len(filenames)}: {fname}")
        stl_path = Path(stl_dir) / fname
        fname_split = fname[:-4].split("-")
        patient_ID = ("-").join(fname_split[:5])
        series_UID = fname_split[-1]
        ROI = fname_split[5]
        ref_img_path = Path(img_dir) / patient_ID / (series_UID + ".nii.gz")
        if ref_img_path.exists():
            (Path(out_dir) / patient_ID).mkdir(exist_ok=True, parents=True)
            output_path = Path(out_dir) / patient_ID / (f"{ROI}_{series_UID}.nii.gz")
            if not output_path.exists():
                stl_to_nifti(stl_path, ref_img_path, output_path)


convert_segmentations(
    stl_dir="/stl",
    img_dir="/nifti/img",
    out_dir="/nifti/seg",
)
